In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import datetime

<img src="optimizer_graph.png" width="200px">

## Reading Data

Data from https://archive.ics.uci.edu/ml/datasets/auto+mpg

In [ ]:
autompg = pd.read_csv('auto_mpg.csv')
# Extract only the displacement and mpg column
autompg_disp = autompg['displacement'].astype(float)
autompg_mpg = autompg['mpg'].astype(float)

# Scale data
autompg_disp = autompg_disp.apply(lambda x:x/100)
print(autompg_disp.describe())
autompg_mpg = autompg_mpg.apply(lambda x: x/100)
print(autompg_mpg.describe())

In [ ]:
# Convert numpy data series to numpy array
train_X = np.asarray(autompg_disp)
train_Y = np.asarray(autompg_mpg)
n_samples = train_X.shape[0]

In [ ]:
# get batch of data
def getbatch(xval, yval, arraylength, batchsize=30):
    count = 0 
    while count < arraylength/batchsize:
        randstart = random.randint(0, arraylength-batchsize-1)
        count += 1
        yield (xval[randstart:randstart+batchsize], yval[randstart:randstart+batchsize])

# Test
#for i in getbatch(train_X, train_Y, n_samples):
#    print(i)

## Setting hyper-parameters

In [ ]:
learning_rate = 0.003
n_epochs = 3000
display_step = 100

## Defining X and Y as placeholders

In [ ]:
# Define placeholders and variables
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

## Initializing Weights and Biases

In [ ]:
W = tf.Variable(np.random.randn())
b = tf.Variable(np.random.randn())

## Defining model

predicted is X*W+b

In [ ]:
pred = tf.add(tf.multiply(X, W), b)

## Define cost and optimizer

In [ ]:
# The cost function is ((predicted-actual)^2)/2*n_samples. 
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Add all ops that need to be initialized
# The initilization needs to be run only after session is created
# as in session.run(init) below.
init = tf.global_variables_initializer()

In [ ]:
# This cell is new compared to the previous linear regression code
# Summary writer and its configuration
writer = tf.summary.FileWriter("./logs", graph=tf.get_default_graph())
tf.summary.scalar("Cost", cost)
merged_summary_op = tf.summary.merge_all()

## Running the session 
This will run the graph and use all the tensors that were previously defined

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for (x, y) in getbatch(train_X, train_Y, n_samples):
            # In this line, we added write merged_summary that will write the cost to tensorboard
            _, costval, merged_summary = sess.run([optimizer, cost, merged_summary_op], 
                                                  feed_dict={X: x, Y: y})
        writer.add_summary(merged_summary, epoch)
        # Every few display step, calculate and print cost
        if (epoch+1) % display_step == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            print("Epoch = {:04d} and cost = {:.9f}".format(epoch+1, c))

    # At the end of the training, print the cost and W and b
    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    finalW = sess.run(W)
    finalb = sess.run(b)
    print("The final W = %0.4f and b = %0.4f" %(finalW, finalb))
    
writer.close()

## Opening tensorboard

In the command line, type "tensorboard --logdir=./logs"
This will launch a local webserver at port 6006. Visit http://localhost:6006 in a browser. Then click on the label 'Cost' to see the graph.  You can make the graph larger by clicking on the rectangular button at the bottom left corner of the graph. You can zoom in to the graph by drawing a box around the region to zoom in. You can zoom out by double clicking the graph.

NOTE 1: Please make sure that you ran this code to completion and that the log folder exist. 

NOTE 2: Make sure that you are in the directory that has the logs folder when you execute the tensorboard command. 

On Windows, visiting http://localhost:6006 returns an error. Instead use the link provided by Tensorboard in the terminal. On the other hand, some people using Mac have experienced problem using the link provided by Tensorboard.  In such cases, use localhost:6006 instead. 